In [ ]:
#Please enter here the paths of the training file and the test file
training_file = "/kuacc/users/bozyurt20/seq2tree_geoqueries/train.txt"
test_file = "/kuacc/users/bozyurt20/seq2tree_geoqueries/test.txt"

In [ ]:
using Knet: Knet, AutoGrad, param, param0, mat, RNN, relu, Data, adam, progress, nll, zeroone
using Distributions
import .Iterators: cycle, Cycle, take
using IterTools
import CUDA
using CUDA

In [ ]:
function tok_int(training_file)
    
    #global int2tok_input, int2tok_output
    f = open(training_file)
    tok2int_input = Dict{String,Int}() #keys: unique input tokens. values: id number of the token.
    int2tok_input = Vector{String}() #indices: numbers. values: tokens corresponding to those numbers.
    push!(int2tok_input, "<s>") #start token
    push!(int2tok_input, "</s>") #stop token
    push!(int2tok_input, "<n>") 
    push!(int2tok_input, "(") 
    push!(int2tok_input, ")") 
    push!(int2tok_input, "UNK") 
    tok2int_input["<s>"] = 1
    tok2int_input["</s>"] = 2
    tok2int_input["<n>"] = 3
    tok2int_input["("] = 4
    tok2int_input[")"] = 5
    tok2int_input["UNK"] = 6
    tok2int_output = Dict{String,Int}() #keys: unique output tokens. values: id number of the token.
    int2tok_output = Vector{String}() #keys: numbers. values: tokens corresponding to those numbers.
    push!(int2tok_output, "<s>") #start token
    push!(int2tok_output, "</s>") #stop token
    push!(int2tok_output, "<n>")
    push!(int2tok_output, "(")
    push!(int2tok_output, ")")
    push!(int2tok_output, "UNK") 
    tok2int_output["<s>"] = 1
    tok2int_output["</s>"] = 2
    tok2int_output["<n>"] = 3
    tok2int_output["("] = 4
    tok2int_output[")"] = 5
    tok2int_output["UNK"] = 6
    while ! eof(f)
        seq = readline(f)
        seq = chomp(seq)
        input, output = split(seq, "\t")
        tokens = split(input, " ")
        for token in tokens
            if !haskey(tok2int_input, token)
                push!(int2tok_input, token)
                tok2int_input[token] = length(int2tok_input)
            end            
        end
        tokens = split(output, " ")
        for token in tokens
            if !haskey(tok2int_output, token)
                push!(int2tok_output, token)
                tok2int_output[token] = length(int2tok_output)
            end           
        end        
    end
    Vq = length(int2tok_input) #number of unique input tokens
    Va = length(int2tok_output) #number of unique output tokens    
    return int2tok_input, tok2int_input, int2tok_output, tok2int_output, Va, Vq
end

In [ ]:
function data_reader(training_file, tok2int_input, tok2int_output)
    
    data = []
    f = open(training_file)
    while ! eof(f)
        seq = readline(f)
        seq = chomp(seq)
        input, output = split(seq, "\t")
        tokens = split(input, " ")
        s = Vector{Int}() #vector that stores the token ids.
        global n_in = 0 #just to check the number of unknown tokens
        for token in tokens            
            if haskey(tok2int_input, token)
                push!(s, tok2int_input[token])
            else
                push!(s, tok2int_input["UNK"])
                n_in += 1
            end 
        end        

        tokens = split(output, " ")
        s2 = Vector{Int}() #vector that stores the token ids.
        global n_out = 0 #just to check the number of unknown tokens
        for token in tokens
            if haskey(tok2int_output, token)
                push!(s2, tok2int_output[token])
            else
                push!(s2, tok2int_output["UNK"])
                n_out += 1
            end
        end 
        t = convert_to_tree(s2, 1, length(s2))
        push!(data, (s, s2, t))
    end    
    return data
end

In [ ]:
function minibatch(data,batchsize)
    n = length(data)
    p = 0
    batch_data = Any[]
    while p + batchsize <= n
        max_seq_len = length(data[p+batchsize][1])
        enc_seq = zeros(Int64, batchsize, max_seq_len+2) 
        enc_seq[:,max_seq_len+2] .= 2 #end of sequence token
        for i in 1:batchsize
            seq = data[p+i][1]
            seq_len = length(seq)
            for j in 1:seq_len
                enc_seq[i,max_seq_len-seq_len+j+1] = seq[seq_len-j+1]
            end
            for k in 1:max_seq_len+1-seq_len #pad the start with start of seq tokens
                enc_seq[i,k] = 1
            end
        end
        
        dec_seq = Any[]
        for i in 1:batchsize
            push!(dec_seq, data[p+i][3])
        end        
        
        p += batchsize
        push!(batch_data, (enc_seq, dec_seq))
    end
    return batch_data
end  

In [ ]:
mutable struct Tree
    parent
    num_children::Int
    children
end

Tree() = Tree(nothing, 0, Any[])

function add_child(t::Tree, c)
    if typeof(c) == Tree
        t.parent = c
    end
    push!(t.children,c)
    t.num_children = t.num_children + 1
end

function convert_to_tree(seq, i_left, i_right)
    t = Tree()
    level = 0
    left = 0
    for i in i_left:i_right
        if seq[i] == tok2int_output["("]
            if level == 0
                left = i
            end
            level = level + 1
        elseif seq[i] == tok2int_output[")"]
            level = level -1
            if level == 0
                if i == left+1
                    c = seq[i]
                else
                    c = convert_to_tree(seq, left + 1, i)
                end
                add_child(t,c)
            end
        elseif level == 0
            add_child(t, seq[i])
        end
    end
        
    return t
end

function to_list(t::Tree, tok2int) 
    r_list = []
    for i in 1:t.num_children
        if typeof(t.children[i]) == Tree
            push!(r_list, tok2int["("])
            cl = to_list(t.children[i], tok2int)
            for k in 1:length(cl)
                push!(r_list, cl[k])
            end
            push!(r_list, tok2int[")"])
        else
            push!(r_list, t.children[i])
        end
    end
    return r_list
end

In [ ]:
function mask(a, pad)
    a = copy(a)
    for i in 1:size(a, 1)
        j = size(a,2)
        while a[i, j] == pad && j > 1
            if a[i, j - 1] == pad
                a[i, j] = 0
            end
            j -= 1
        end
    end
    return a
end

In [ ]:
struct Embed
    w
end
function Embed(embedsize::Int, vocabsize::Int)
    w = rand(Uniform(-0.08,0.08),embedsize,vocabsize)
    w = Knet.Param(convert(Knet.KnetArray{Float32},w))
    return Embed(w)
end
(e::Embed)(x) = e.w[:,x] #x: word id

struct Linear
    w
    b
    f
end

#Linear(i::Int,o::Int,f=identity) = Linear(param(o,i), param0(o), f)

function Linear(i::Int,o::Int,f=identity)
    w = rand(Uniform(-0.08,0.08),o,i)
    w = Knet.Param(convert(Knet.KnetArray{Float32},w))
    b = param0(o)
    return Linear(w,b,f)
end

(d::Linear)(x) = d.f.(d.w * mat(x,dims=1) .+ d.b)


In [ ]:
struct seq2tree
    input_embed
    output_embed
    encoder
    decoder
    linear
    dropout
end

function seq2tree(X::Int, H::Int, Vq::Int, Va::Int, dropout::Real)
    a = Embed(X, Vq)
    b = Embed(X, Va)
    c = RNN(X, H; rnnType=:lstm, numLayers=1, dropout=dropout)
    d = RNN(X, H; rnnType=:lstm, numLayers=1, dropout=dropout)
    #d = RNN((X+H), H; rnnType=:lstm, numLayers=1, dropout=dropout)
    e = Linear(H,Va)
    f = dropout
    return seq2tree(a, b, c, d, e, f)
end

function (s::seq2tree)(x, y; average=true) 
    # x: B,Tmaxx  y: B 
    batchsize = size(x, 1)
    
    s.encoder.h, s.encoder.c = 0, 0
    x_embedded = s.input_embed(x); @assert size(x_embedded)==(X,batchsize,size(x,2)) # X,B,Tmaxx
    encoder_out = s.encoder(x_embedded); @assert size(encoder_out)==(X,batchsize,size(x,2)) # H,B,Tmaxx
    hiddensize = size(encoder_out, 1)
    dec_s = Dict()
    for i in 0:120
        dec_s[i] = Dict()
        for j in 0:120
            dec_s[i][j] = Dict()
        end
    end
    
    queue_tree = Dict()    
    
    for i in 1:batchsize
        queue_tree[i] = Any[]
        push!(queue_tree[i], Dict("tree" => y[i], "parent"=> 0, "child_index"=> 1))
    end
    loss = 0
    cur_index, max_index = 1, 1
    dec_batch = Dict()
    while (cur_index <= max_index)
        max_w_len = -1
        batch_w_list = Any[]
        for i in 1:batchsize
            w_list = Any[]
            if (cur_index <= length(queue_tree[i]))
                t = queue_tree[i][cur_index]["tree"]
                for ic in 1:t.num_children
                    if typeof(t.children[ic]) == Tree
                        push!(w_list, tok2int_output["<n>"])
                        push!(queue_tree[i], Dict("tree" => t.children[ic], "parent" => cur_index, "child_index"=> ic)) 
                    else
                        push!(w_list, t.children[ic])
                    end
                end
                if length(queue_tree[i]) > max_index
                    max_index = length(queue_tree[i])
                end
            end
            if length(w_list) > max_w_len
                max_w_len = length(w_list)
            end
            push!(batch_w_list, w_list)
        end
        #dec_batch: max_cur_index, B, Tmaxx +2
        #dec_batch[cur_index] = zeros(batchsize, max_w_len+2)
        dec_batch[cur_index] = fill(tok2int_output["</s>"], (batchsize, max_w_len+2)) 
        for i in 1:batchsize 
            w_list = batch_w_list[i]
            if length(w_list) > 0
                for j in 1:length(w_list)
                    dec_batch[cur_index][i,j+1] = w_list[j]
                end
                
                # add start and end tokens
                if cur_index == 1
                    dec_batch[cur_index][i,1] = tok2int_output["<s>"]
                else
                    dec_batch[cur_index][i,1] = tok2int_output["("]
                end
                dec_batch[cur_index][i,length(w_list) + 2] = tok2int_output["</s>"]
            end
        end
        #println(dec_batch[cur_index])
        dec_s[cur_index][0][1] = convert(Knet.KnetArray{Float32}, zeros(hiddensize, batchsize, 1)) 
        dec_s[cur_index][0][2] = convert(Knet.KnetArray{Float32}, zeros(hiddensize, batchsize, 1))
        
        if cur_index == 1
            """
            for i in 1:batchsize
                dec_s[1][0][1][i, :] = s.encoder.c()
                dec_s[1][0][2][i, :] = s.encoder.h()
            end
            """
            dec_s[1][0][1] = s.encoder.c
            dec_s[1][0][2] = s.encoder.h
        else
            for i in 1:batchsize
                if (cur_index <= length(queue_tree[i]))
                    par_index = queue_tree[i][cur_index]["parent"]
                    child_index = queue_tree[i][cur_index ]["child_index"]
                    dec_s[cur_index][0][1][:,i,1] .= Knet.value(dec_s[par_index][child_index][1][:,i,1])
                    dec_s[cur_index][0][2][:,i,1] .= Knet.value(dec_s[par_index][child_index][2][:,i,1])
                end
            end
        end
        labels = dec_batch[cur_index][:,2]
        for i in 1:size(dec_batch[cur_index], 2)-1
            s.decoder.c = dec_s[cur_index][i-1][1]
            s.decoder.h = dec_s[cur_index][i-1][2]
            y_embedded = s.output_embed(dec_batch[cur_index][:,i]); @assert size(y_embedded)==(X,batchsize) # X,B
            """
            if cur_index == 1
                nonterminal = fill(tok2int_input["</s>"], (batchsize)) # B
                embed_nonterminal = s.input_embed(nonterminal); @assert size(embed_nonterminal)==(H,batchsize) # X,B
            else
                nonterminal = fill(tok2int_output["<n>"], (batchsize)); @assert size(nonterminal)==(batchsize,)# B
                embed_nonterminal = s.output_embed(nonterminal); @assert size(embed_nonterminal)==(H,batchsize) # X,B
            end
            
            decoder_in = vcat(y_embedded, embed_nonterminal); @assert size(decoder_in)==(2*H,batchsize) # 2X,B
            decoder_out = s.decoder(decoder_in); @assert size(decoder_out)==(H,batchsize) # H,B
            """
            decoder_out = s.decoder(y_embedded); @assert size(decoder_out)==(H,batchsize) # H,B
            dec_s[cur_index][i][1] = s.decoder.c
            dec_s[cur_index][i][2] = s.decoder.h
            #dims = size(decoder_out) # H,B
            scores = s.linear(Knet.dropout(decoder_out, s.dropout)); @assert size(scores)==(Va,batchsize) #Va, B
            
            
            #loss += nll(scores, Int.(dec_batch[cur_index][:,i+1]); dims=1, average=average)
            loss += nll(scores, labels; dims=1, average=average)
            
            if i != size(dec_batch[cur_index], 2)-1
                for j in 1:batchsize
                    if (dec_batch[cur_index][j,i+1]) == 2
                        labels[j] = 0
                    else
                        labels[j] = Int(dec_batch[cur_index][j,i+2]) 
                    end
                end
            end
            
                
        end
        cur_index += 1
    end
    loss = loss / batchsize
    #println(loss)
    return loss

    
end

In [ ]:
### FOR PREDICTION/TESTING
function (s::seq2tree)(x_data)
    x = copy(x_data)
    push!(x, tok2int_input["<s>"])
    pushfirst!(x, tok2int_input["</s>"])
    x = reverse(x) # T 
    #println(x)
    x = reshape(x, 1, length(x)) # 1,T
    #println(x)
    x_embedded = s.input_embed(x); @assert size(x_embedded)==(X,1,length(x)) # X,1,T 
    s.encoder.h, s.encoder.c = 0, 0
    embedded_out = s.encoder(x_embedded); @assert size(embedded_out)==(H,1,length(x))# H,1,T
    #println(embedded_out)
    
    
    queue_decode = Any[] 
    push!(queue_decode, Dict("ch" => (s.encoder.c, s.encoder.h), "parent"=>0, "child_index"=>1, "t"=> Tree()))
    
    
    
    head = 1
    while head <= length(queue_decode) && head <= 100
        ch = queue_decode[head]["ch"]
        t = queue_decode[head]["t"]
        
        #h = copy(ch[2])
        #h = reshape(h, size(h,1)); @assert size(h)==(H,)
        
        if head == 1
            prev_word = [tok2int_output["<s>"]] 
        else
            prev_word = [tok2int_output["("]] 
        end
        
        i_child = 1
        while true
            prev_word_embed = s.output_embed(prev_word[1]); @assert size(prev_word_embed)==(X,) # X
            
            #decoder_in = vcat(prev_word_embed, h); @assert size(decoder_in)==((X+H),) # 2X
            s.decoder.c, s.decoder.h = ch[1], ch[2]
            #decoder_out = s.decoder(decoder_in); @assert size(decoder_out)==(H,) # H
            decoder_out = s.decoder(prev_word_embed); @assert size(decoder_out)==(H,) # H
            ch = (s.decoder.c, s.decoder.h)
            output = s.linear(decoder_out); @assert size(output)==(Va, 1) # Va,1
            prev_word = argmax(output)
            if Int(prev_word[1]) == tok2int_output["</s>"] || t.num_children >= 120 
                #println("if",prev_word[1])
                break
            elseif Int(prev_word[1]) == tok2int_output["<n>"]                 
                push!(queue_decode, Dict("ch" => (copy(ch[1]), copy(ch[2])), "parent"=>head, "child_index"=>i_child, "t"=> Tree()))                
                add_child(t, prev_word[1])
            else
                add_child(t, prev_word[1])
            end
            i_child += 1
        end
        
        head += 1
    end
    
    for i in length(queue_decode):-1:2
        cur = queue_decode[i]
        queue_decode[cur["parent"]]["t"].children[cur["child_index"]] = cur["t"] 
    end
    return to_list(queue_decode[1]["t"], tok2int_output)
    
    
    
end

In [ ]:
function model_accuracy(model, data)
    total = 0
    num_sequences = 0
    for (x, y, t) in data
        
        num_sequences += 1
        
        y_pred = model(x) # x: Tx y: Ty 
     
        
        num_left_paren = sum([1 for c in y_pred if int2tok_output[c] == "("])    
        num_right_paren = sum([1 for c in y_pred if int2tok_output[c]== ")"])
                                
        diff = num_left_paren - num_right_paren
        if diff > 0
            for i in 1:diff
                push!(y_pred, tok2int_output[")"])
            end
        elseif diff < 0
            y_pred_clipped = y_pred[1:length(y_pred)+diff]
        end
                                
        if length(y) == length(y_pred)
            if sum(y_pred .== y) == length(y)                 
                total += 1                    
            end
        end
        
    end
    return total / num_sequences
end

In [ ]:
lrdecay!(s::seq2tree, decay::Real) =
    for p in Knet.params(s); p.opt.lr = p.opt.lr*decay; end

In [ ]:
int2tok_input, tok2int_input, int2tok_output, tok2int_output, Va, Vq = tok_int(training_file)
trndata = data_reader(training_file, tok2int_input, tok2int_output)
tstdata = data_reader(test_file, tok2int_input, tok2int_output)
X = 200
H = 200 
trndata_batch = minibatch(trndata, 20)

In [ ]:
my_model = seq2tree(X,H,Vq,Va,0.4)

In [ ]:
Knet.progress!(Knet.rmsprop(my_model, ncycle(trndata_batch,4); rho=0.95, gclip=5, lr = 0.005))

for i in 5:200
    
    #println(Knet.params(my_model)[1].opt.lr)
    Knet.rmsprop!(my_model, trndata_batch;rho=0.95,gclip=5)
    lrdecay!(my_model, 0.97)
    if i == 20
        for p in Knet.params(my_model); p.opt.lr = 0.005; end
    end
    
    if i%5 == 0
        println("After $i epochs")
        println("train accuracy: ", model_accuracy(my_model, trndata))
        println("test accuracy: ", model_accuracy(my_model, tstdata))
    end
end
# Knet.progress!(Knet.rmsprop(my_model, ncycle(trndata_batch, 50);rho=0.95,gclip=5))